In [2]:
!pip install -q -U watermark

Импортируем нужные библиотеки:

In [3]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, pearsonr, spearmanr, \
    kendalltau, f_oneway, kruskal

Версии пакетов, используемых в этом Jupiter Netbook

In [4]:
%reload_ext watermark
%watermark -a "Гасымов Равиль" --iversions

Author: Гасымов Равиль

numpy      : 1.16.2
seaborn    : 0.9.0
pandas     : 0.24.2
statsmodels: 0.9.0
matplotlib : 3.0.3



# **Анализ** **исследования:**


userid: уникальный номер, который идентифицирует каждого игрока.

version = Был ли игрок поставлен в контрольную группу (gate_30 - ворота 30 уровня) или в группу с перемещенными воротами (gate_40 - ворота 40 уровня).

sumgamerounds = количество раундов, сыгранных каждым игроком в течение первых 14 дней после установки.

retention_1 = Вернулся ли игрок в игру через 1 день после установки.

retention_7 = Вернулся ли игрок в игру через 7 дней после установки.

In [6]:
df = pd.read_csv(r"cookie_cats.csv", encoding= 'unicode_escape')
df

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
5,540,gate_40,187,True,True
6,1066,gate_30,0,False,False
7,1444,gate_40,2,False,False
8,1574,gate_40,108,True,True
9,1587,gate_40,153,True,False


In [7]:
df.shape

(90189, 5)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
userid            90189 non-null int64
version           90189 non-null object
sum_gamerounds    90189 non-null int64
retention_1       90189 non-null bool
retention_7       90189 non-null bool
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [9]:
df.isnull().sum()

userid            0
version           0
sum_gamerounds    0
retention_1       0
retention_7       0
dtype: int64

Удержание в течении 1 дня после установки:


In [10]:
df.retention_1.value_counts()

False    50036
True     40153
Name: retention_1, dtype: int64

Удержание в течении 7 дней после установки:

In [11]:
df.retention_7.value_counts()

False    73408
True     16781
Name: retention_7, dtype: int64

In [12]:
df.shape

(90189, 5)

In [14]:
def check_df(dataframe, head=5):
    print("########## Info #############")
    print(dataframe.info())
    print("########## Shape #############")
    print(dataframe.shape)
    print("########## Data Types #############")
    print(dataframe.dtypes)
    print("########## Head of Data #############")
    print(dataframe.head(head))
    print("########## Tail of Data #############")
    print(dataframe.tail(head))
    print("########## Null Values of Data #############")
    print(dataframe.isnull().sum())
    print("########## Describe of the Numerical Datas #############")
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

########## Info #############
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
userid            90189 non-null int64
version           90189 non-null object
sum_gamerounds    90189 non-null int64
retention_1       90189 non-null bool
retention_7       90189 non-null bool
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB
None
########## Shape #############
(90189, 5)
########## Data Types #############
userid             int64
version           object
sum_gamerounds     int64
retention_1         bool
retention_7         bool
dtype: object
########## Head of Data #############
   userid  version  sum_gamerounds  retention_1  retention_7
0     116  gate_30               3        False        False
1     337  gate_30              38         True        False
2     377  gate_40             165         True        False
3     483  gate_40               1        False        False
4     488  gate_40             179         T

Перед началом анализа, согласно сводке набора данных, набор данных имеет 5 переменных. Давайте проверим их:

userid: уникальный номер, который идентифицирует каждого игрока.

version: Был ли игрок поставлен в контрольную группу (gate_30 - ворота 30 уровня) или в группу с перемещенными воротами (gate_40 - ворота 40 уровня).

sum_gamerounds: количество игровых раундов, сыгранных игроком в течение первых 14 дней после установки.

retention_1: Игрок вернулся через 1 день после установки?

retention_7: Игрок вернулся через 7 дней после установки?




Согласно быстрому анализу данных, в наборе данных нет нулевых данных, и все идентификаторы пользователей используются по отдельности.

**Подготовка данных:**

На этом этапе, если в наборе данных есть нулевые значения, то они удалятся из данных.

In [15]:
# Команда dropna() поможет удалить нулевые значения из данных.

df.shape
df.isnull().sum()
df.dropna(inplace = True)

# Для проверки данных

df.describe().T

,count,mean,std,min,25%,50%,75%,max
userid,90189.0,4.998412e+06,2.883286e+06,116.0,2512230.0,4995815.0,7496452.0,9999861.0
sum_gamerounds,90189.0,5.187246e+01,1.950509e+02,0.0,5.0,16.0,51.0,49854.0


**Нормальное распределение:**

H0: Верно предположение о нормальном распределении.
H1: Ложь.

Если p-значение < 0,05, то мы отклоняем нулевую гипотезу в пользу альтернативной.

Если p-значение > 0,05, то тогда мы принимаем нулевую гипотезу.



 Контроль нормального распределения для gate_30:

In [16]:
test_stat, pvalue = shapiro(df.loc[df["version"] == "gate_30", "sum_gamerounds"])
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

Test Stat = 0.0881, p-value = 0.0000


F:\Anaconda3\lib\site-packages\scipy\stats\morestats.py:1653: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


Контроль нормального распределения для gate_40:

In [17]:
test_stat, pvalue = shapiro(df.loc[df["version"] == "gate_40", "sum_gamerounds"])
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

Test Stat = 0.4826, p-value = 0.0000


F:\Anaconda3\lib\site-packages\scipy\stats\morestats.py:1653: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


Вывод:

Поскольку для обеих групп (для gate_30 и gate_40 p-value<0.05, то мы можем сделать следующее заключение:

НЕ верно предположение о нормальном распределении, для обоих групп.

**Допущение однородности дисперсии:**

Предположение об однородности дисперсии означает, что уровень дисперсии для конкретной переменной постоянен по всей выборке. Если мы собрали группы данных, это означает, что дисперсия нашей переменной (переменных) результата должна быть одинаковой в каждой из этих групп (т.е. по группам тестирования или прогнозируемым значениям).

Предположение об однородности дисперсии:

H0: Дисперсия однородна.

H1: Дисперсия не является однородной.

Если p-value < 0,05, то мы отвергаем нулевую гипотезу в пользу альтернативной

Если p-value > 0,05, то тогды мы принимаем нулевую гипотезу.

In [18]:
test_stat, pvalue = levene(df.loc[df["version"] == "gate_30", "sum_gamerounds"],
                            df.loc[df["version"] == "gate_40", "sum_gamerounds"])
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

Test Stat = 0.5292, p-value = 0.4669


**Вывод**:

так как p-value >0.05, то дисперсия однородна.

**Применение гипотез:**

Согласно «Нормальному распределению» и «Предположению об однородности дисперсии» показано, что первый контроль НЕ принимает нулевую гипотезу, а второй принимает её. Это означает, что мы должны использовать команду mannwtihneyu() для применения гипотезы.

Критерий Манна-Уитни основан на сравнении каждого наблюдения xi в первой выборке с каждым наблюдением yj в другой выборке. Общее количество попарных сравнений, которые можно сделать, равно nxny.1

In [19]:
test_stat, pvalue = mannwhitneyu(df.loc[df["version"] == "gate_30", "sum_gamerounds"],
                            df.loc[df["version"] == "gate_40", "sum_gamerounds"])
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

Test Stat = 1009027049.5000, p-value = 0.0251


**Вывод:**

Так как p-value >0.05 то мы приманием нулевую гипотезу  о равенстве средних значений двух выборок.( т.е. между gate_30 и gate_40)

**ОТВЕТ**:

Заключение и рекомендация.

Согласно результатам A/B-тестирования, чтобы выяснить, как «gate_30» и «gate_40» влияют на общее количество игровых раундов, сыгранных в игре?

В первом процессе тестирования мы спрашиваем, как «средства равны или нет» для примера, описанного выше. Чтобы выяснить это, сначала проверили нормальное распределение и предположение об однородности дисперсии, распределены ли они нормально или нет? A/B-тестирование говорит, что если нормальное распределение не равно, вам следует использовать U-критерий Манна-Уитни, чтобы выяснить влияние «gate_30» и «gate_40».

Результат U-критерия Манна-Уитни, p-value рассчитано как 0,0502, и это означает, что нулевую невозможно отклонить в соответствии с предположением гипотезы, и его значение для A/B-тестирования, результаты для обоих групп(gate_30 и gate_40) были получены случайно.

Исходя из результатов эксперимента, мы не можем однозначно сказать, что использование «gate_30» или «gate_40» влияет на общее количество игровых раундов. Мы не можем отвергнуть гипотезу, что различия между группами случайны.

Наша рекомендация для менеджера заключается в том, чтобы продолжать мониторить результаты и дополнительные данные эксперимента. Можно провести более широкие и продолжительные тесты, чтобы более полно оценить потенциальное влияние каждого «гейта» на игровой процесс. Также можно более подробно рассмотреть конкретные пользовательские данные, чтобы понять, есть ли какие-то сегменты пользователей, где один из «гейтов» работает лучше, чем другой.